In [1]:
import os
import pathlib
import sys
import time
import urllib

import numpy as np
import pandas as pd
import psutil
from arg_parsing_utils import check_for_missing_args, parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

from loading_classes import ImageSetLoader, ObjectLoader
from resource_profiling_util import get_mem_and_time_profiling
from sammed3d_featurizer import call_SAMMed3D_pipeline

In [2]:
if not in_notebook:
    arguments_dict = parse_args()
    patient = arguments_dict["patient"]
    well_fov = arguments_dict["well_fov"]
    compartment = arguments_dict["compartment"]
    channel = arguments_dict["channel"]

else:
    well_fov = "C4-2"
    patient = "NF0014_T1"
    compartment = "Organoid"
    channel = "DNA"

image_set_path = pathlib.Path(f"{root_dir}/data/{patient}/zstack_images/{well_fov}/")

output_parent_path = pathlib.Path(
    f"{root_dir}/data/{patient}/extracted_features/{well_fov}/"
)
output_parent_path.mkdir(parents=True, exist_ok=True)

In [3]:
sam3dmed_checkpoint_url = (
    "https://huggingface.co/blueyo0/SAM-Med3D/resolve/main/sam_med3d_turbo.pth"
)
sam3dmed_checkpoint_path = pathlib.Path("../models/sam-med3d-turbo.pth").resolve()
if not sam3dmed_checkpoint_path.exists():
    sam3dmed_checkpoint_path.parent.mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(sam3dmed_checkpoint_url, str(sam3dmed_checkpoint_path))

In [4]:
channel_n_compartment_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [5]:
start_time = time.time()
# get starting memory (cpu)
start_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [6]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_n_compartment_mapping,
)

In [7]:
# load the objects for the compartment and channel of interest
object_loader = ObjectLoader(
    image_set_loader.image_set_dict[channel],
    image_set_loader.image_set_dict[compartment],
    channel,
    compartment,
)

feature_dict = call_SAMMed3D_pipeline(
    object_loader=object_loader,
    SAMMed3D_model_path=str(sam3dmed_checkpoint_path),
    feature_type="cls",
)

✓ Loading SAM-Med3D via MedIM from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
creating model SAM-Med3D
try to load pretrained weights from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
✓ Successfully loaded pretrained SAM-Med3D-turbo


In [8]:
final_df = pd.DataFrame(feature_dict)

final_df["feature_name"] = (
    final_df["feature_name"] + "_" + final_df["compartment"] + "_" + final_df["channel"]
)
final_df["feature_name"] = final_df["feature_name"].str.replace("_feature_", ".")
final_df = final_df.drop(columns=["compartment", "channel"])
final_df.head()

,object_id,feature_name,value,feature_type
0,32,SAMMed3D.0_Organoid_DNA,-0.400578,cls
1,32,SAMMed3D.1_Organoid_DNA,-0.319717,cls
2,32,SAMMed3D.2_Organoid_DNA,-0.064227,cls
3,32,SAMMed3D.3_Organoid_DNA,0.011630,cls
4,32,SAMMed3D.4_Organoid_DNA,0.034913,cls


In [9]:
final_df = final_df.pivot(
    index="object_id", columns="feature_name", values="value"
).reset_index()
# drop the multiindexing from pivot
final_df.columns.name = None
final_df

,object_id,SAMMed3D.0_Organoid_DNA,SAMMed3D.100_Organoid_DNA,SAMMed3D.101_Organoid_DNA,SAMMed3D.102_Organoid_DNA,SAMMed3D.103_Organoid_DNA,SAMMed3D.104_Organoid_DNA,SAMMed3D.105_Organoid_DNA,SAMMed3D.106_Organoid_DNA,SAMMed3D.107_Organoid_DNA,...,SAMMed3D.91_Organoid_DNA,SAMMed3D.92_Organoid_DNA,SAMMed3D.93_Organoid_DNA,SAMMed3D.94_Organoid_DNA,SAMMed3D.95_Organoid_DNA,SAMMed3D.96_Organoid_DNA,SAMMed3D.97_Organoid_DNA,SAMMed3D.98_Organoid_DNA,SAMMed3D.99_Organoid_DNA,SAMMed3D.9_Organoid_DNA
0,32,-0.400578,0.017122,-0.118976,0.10289,0.06197,-0.021076,-0.17842,-0.038834,-0.086486,...,-0.088321,0.092049,-0.01041,0.03833,0.039381,-0.02262,0.258761,0.245225,0.244523,0.204665


In [10]:
# prepend compartment and channel to column names
for col in final_df.columns:
    if col not in ["object_id"]:
        final_df[col] = final_df[col].astype(np.float32)

In [11]:
# de-fragment
final_df = final_df.copy()
final_df.insert(1, "image_set", image_set_loader.image_set_name)

output_file = pathlib.Path(
    output_parent_path / f"SAMMed3D_{compartment}_{channel}_features.parquet"
)
final_df.to_parquet(output_file, index=False)
final_df.head()

,object_id,image_set,SAMMed3D.0_Organoid_DNA,SAMMed3D.100_Organoid_DNA,SAMMed3D.101_Organoid_DNA,SAMMed3D.102_Organoid_DNA,SAMMed3D.103_Organoid_DNA,SAMMed3D.104_Organoid_DNA,SAMMed3D.105_Organoid_DNA,SAMMed3D.106_Organoid_DNA,...,SAMMed3D.91_Organoid_DNA,SAMMed3D.92_Organoid_DNA,SAMMed3D.93_Organoid_DNA,SAMMed3D.94_Organoid_DNA,SAMMed3D.95_Organoid_DNA,SAMMed3D.96_Organoid_DNA,SAMMed3D.97_Organoid_DNA,SAMMed3D.98_Organoid_DNA,SAMMed3D.99_Organoid_DNA,SAMMed3D.9_Organoid_DNA
0,32,C4-2,-0.400578,0.017122,-0.118976,0.10289,0.06197,-0.021076,-0.17842,-0.038834,...,-0.088321,0.092049,-0.01041,0.03833,0.039381,-0.02262,0.258761,0.245225,0.244523,0.204665


In [12]:
end_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2
end_time = time.time()
get_mem_and_time_profiling(
    start_mem=start_mem,
    end_mem=end_mem,
    start_time=start_time,
    end_time=end_time,
    feature_type="AreaSizeShape",
    well_fov=well_fov,
    patient_id=patient,
    channel="DNA",
    compartment=compartment,
    CPU_GPU="GPU",
    output_file_dir=pathlib.Path(
        f"{root_dir}/data/{patient}/extracted_features/run_stats/{well_fov}_AreaSizeShape_DNA_{compartment}_GPU.parquet"
    ),
)


        Memory and time profiling for the run:

        Patient ID: NF0014_T1

        Well and FOV: C4-2

        Feature type: AreaSizeShape

        CPU/GPU: GPU")

        Memory usage: 1822.53 MB

        Time:

        --- %s seconds --- % 7.308945417404175

        --- %s minutes --- % 0.12181575695673624

        --- %s hours --- % 0.002030262615945604
    


True